# Bacteria model - Vectorisation

## Introduction

This model builds on the work by [Marsland et al. (2019)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1006793). It includes a consumer pool (bacteria) and a resource pool (organic compounds). The bacteria can feed on specific resources only, and the resources consumed by the bacteria can be converted into other resources. This notebook sets out the two ODEs of the model and the vectorised equations.

## Model equations

For detailed model description and explanation of the equations see this [notebook](https://nbviewer.jupyter.org/github/CleggTom/Bacteria_EcoFunc/blob/master/notebooks/Metabolic_model.ipynb) by Tom Clegg who built the initial model. 

### Consumers

For an $i$th consumers with $j$ resources the core consumer equation is:

\begin{equation}
dC_i/dt = C_i\Bigl(\sum_{j=1}^{M}u_{ij}S_j(1-Rg_j-\sum_{k=1}^{M}l_{jk}) - Rm_i\Bigl) \tag{1}
\end{equation}

where $C_i$ is the biomass of consumer $i$, $u_{ij}$ is the preference of consumer $i$ to uptake resource $j$, $S_j$ is the mass or concentration of resources $j$, $Rg_j$ is the loss of consuming resource $j$ to respiration, $l_{jk}$ is the proportion of $j$ taken up by the bacteria that is converted to and released as resource $k$, and $Rm_i$ is the maintenance cost loss term. 

### Resources

For a $j$th resource the core equation is:

\begin{equation}
dS_j/dt = \rho - \sum_{i=1}^{N}\Bigl(C_iu_{ij}S_j-\sum_{k=1}^{M}C_iu_{ij}S_jl_{kj}\Bigl) \tag{2}
\end{equation}

where $\rho$ is the external input of resource $j$ and $l_{kj}$ is the gain and conversion of resource $j$ from $k$.

## Vectorisation

To optimise the model run time and to allow modifications (e.g. immigration of species) the equations should be written in vector and matrix format.

### Consumer vector equation

\begin{equation}
dC/dt = \begin{matrix}
\begin{bmatrix}
    C_1 \\
    C_2 \\
    \vdots \\
    C_N
\end{bmatrix}
\begin{bmatrix}
    \sum_{j=1}^{M}U_{1j}S_jL_j \\
    \sum_{j=1}^{M}U_{2j}S_jL_j \\
    \vdots \\
    \sum_{j=1}^{M}U_{Nj}S_jL_j
\end{bmatrix}
\begin{bmatrix}
    Rm_1 \\
    Rm_2 \\
    \vdots \\
    Rm_N
\end{bmatrix}
\end{matrix}\tag{3}
\end{equation}



where $L_j$ is a column vector of length $M$ defined by:

\begin{equation}
\begin{bmatrix}
    L_1 \\
    L_2 \\
    \vdots \\
    L_M
\end{bmatrix} =
\begin{bmatrix}
    1 - Rg_1 - \sum_{k=1}^{M}l_1k\\
    1 - Rg_2 - \sum_{k=1}^{M}l_2k\\
    \vdots \\
    1 - Rg_M - \sum_{k=1}^{M}l_Mk
\end{bmatrix}\tag{4}
\end{equation}

### Resource vector equation

\begin{equation}
dS/dt = \begin{matrix}
\begin{bmatrix}
    \rho_1 \\
    \rho_2 \\
    \vdots \\
    \rho_M
\end{bmatrix} -
\begin{bmatrix}
    C_1
    C_2
    \cdots
    C_N
\end{bmatrix}
\begin{bmatrix}
    U_{i1}S_1 \\
    U_{i2}S_2 \\
    \vdots \\
    U_{iM}S_M
\end{bmatrix} +
\begin{bmatrix}
    \sum_{i=1}^{N}\sum_{k=1}^{M}C_iU_{i1}S_1l_{1k} \\
    \sum_{i=1}^{N}\sum_{k=1}^{M}C_iU_{i2}S_2l_{2k} \\
    \vdots \\
    \sum_{i=1}^{N}\sum_{k=1}^{M}C_iU_{iM}S_Ml_{Mk}
\end{bmatrix}
\end{matrix}\tag{5}
\end{equation}

where the final vector $\sum_{i=1}^{N}\sum_{k=1}^{M}C_iU_{ij}S_jl_{jk}$ can be expressed using Einstein notation:

\begin{equation}
\sum_{i=1}^{N}\sum_{k=1}^{M}C_iU_{ij}S_jl_{jk} = C_iU_{j}^{i}S_jl_{j}^{k} \tag{6}
\end{equation}

which can be expressed using the `numpy.einsum()` function in Python. The Python code for the vectorised model `bacteria_vector_ein.py` is [here](https://github.com/e-cavan/Bacteria_Model).